# Advent of Code 2024 Day 15

## Part 1

In [2]:
from dataclasses import dataclass, field
from enum import auto, Enum

In [3]:
class Move(Enum):
    UP = auto()
    RIGHT = auto()
    DOWN = auto()
    LEFT = auto()

    @staticmethod
    def from_str(move_str):
        match move_str:
            case "^":
                move = Move.UP
            case ">":
                move = Move.RIGHT
            case "v":
                move = Move.DOWN
            case "<":
                move = Move.LEFT
            case _:
                raise ValueError("invalid move")
        return move

@dataclass
class BotArea:
    moves: list[Move]
    width: int = 0
    height: int = 0
    position: tuple = (0, 0)
    boxes: set[tuple[int, int]] = field(default_factory=set)
    obstacles: set[tuple[int, int]] = field(default_factory=set)
    

    def with_puzzle(self, puzzle_area):
        self.width = len(puzzle_area)
        self.height = len(puzzle_area[0])

        for x, row in enumerate(puzzle_area):
            for y, c in enumerate(row):
                if c == "O":
                    self.boxes.add((x, y))
                elif c == "@":
                    self.position = (x, y)
                elif c == "#" and 1 <= x < self.width - 1 and 1 <= y < self.height- 1:
                    self.obstacles.add((x, y))

        return self

    def sim_moves(self):
        for move in self.moves:
            try:
                self._move_bot(move)
                # print(self.position, move)
            except Exception as e:
                # print(e)
                pass

    def _move_bot(self, move):
        match move:
            case Move.UP:
                delta = (-1, 0)
            case Move.RIGHT:
                delta = (0, 1)
            case Move.DOWN:
                delta = (1, 0)
            case Move.LEFT:
                delta = (0, -1)

        next_pos = (self.position[0] + delta[0], self.position[1] + delta[1])

        if next_pos in self.obstacles or not all([1 <= next_pos[0] < self.width - 1, 1 <= next_pos[1] < self.height - 1]):
            raise Exception(f"cannot move into wall or obstacle => {next_pos}")
        if next_pos in self.boxes:
            self._push_boxes(next_pos, delta)
        self.position = next_pos

    def _push_boxes(self, start_pos, delta):
        current_pos = start_pos
        chain = []

        while current_pos in self.boxes:
            chain.append(current_pos)
            current_pos = (current_pos[0] + delta[0], current_pos[1] + delta[1])

        if not (1 <= current_pos[0] < self.width - 1 and 1 <= current_pos[1] < self.height- 1):
            raise Exception(f"cannot push box into a wall => {current_pos}")

        if current_pos in self.boxes or current_pos in self.obstacles:
            raise Exception(f"cannot push box into another box or obstacle => {current_pos}")

        for pos in reversed(chain):
            next_pos = (pos[0] + delta[0], pos[1] + delta[1])
            # print(f"PUSHED: {pos} -> {next_pos}")
            self.boxes.remove(pos)
            self.boxes.add(next_pos)

        

        

In [4]:
puzzle = []
moves = []
coord_sum = 0

with open("data/gps.txt", "r") as file:
    content = file.readlines()
    line_br = content.index("\n")
    puzzle = [line.strip() for line in content[:line_br]]
    move_strs = "".join([line.strip() for line in content[line_br+1:]])
    moves = [Move.from_str(m) for m in move_strs]

area = BotArea(moves).with_puzzle(puzzle)
area.sim_moves()
coord_sum = sum((100 * box[0]) + box[1] for box in area.boxes)
coord_sum

1442192

In [6]:
with open("answer.txt", "w") as file:
    file.writelines([str(coord_sum)])